In [1]:
import pandas as pd
from time import sleep

from nba_api.stats.static import players
from nba_api.stats.endpoints import playerawards, playercareerstats

In [2]:
#get list of all inactive players; to be used for training/testing
inactives=pd.DataFrame(players.get_inactive_players())
inactives


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4568,1627790,Ante Zizic,Ante,Zizic,False
4569,78647,Jim Zoet,Jim,Zoet,False
4570,78648,Bill Zopf,Bill,Zopf,False
4571,78650,Matt Zunic,Matt,Zunic,False


In [7]:
test_df=inactives.iloc[:5].copy()
test_df

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [ ]:
# This function creates the label variable - whether or not a player is in the HOF
def get_hof(row):
    # sleeping to respect NBA's rate limiting
    sleep(0.5)
    awards = playerawards.PlayerAwards(row["id"]).get_data_frames()[0]
    return "Hall of Fame Inductee" in awards["DESCRIPTION"].values


test_df["HOF"] = test_df.apply(get_hof, axis=1)
test_df

,id,full_name,first_name,last_name,is_active,HOF
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False,True
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False,False


In [29]:
def get_career_stats(row):
    # sleep to respect rate limiting
    sleep(0.5)
    stats=playercareerstats.PlayerCareerStats(row["id"]).get_data_frames()[0]
    #remove total entry to prevent double counting
    stats=stats[stats["TEAM_ABBREVIATION"]!= "TOT"]
    #print(stats)
    return pd.concat([stats.iloc[:,6:11],stats.iloc[:,12:14],stats.iloc[:,15:17],stats.iloc[:,18:]]).sum()
playercareerstats.PlayerCareerStats(76003).get_data_frames()[0].columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [24]:
ai=inactives[inactives["id"]==947]
ai

,id,full_name,first_name,last_name,is_active
1928,947,Allen Iverson,Allen,Iverson,False


In [30]:
get_career_stats(ai)

GP        914.0
GS        901.0
MIN     37582.0
FGM      8467.0
FGA     19906.0
FG3M     1059.0
FG3A     3383.0
FTM      6375.0
FTA      8168.0
OREB      745.0
DREB     2649.0
REB      3394.0
AST      5624.0
STL      1983.0
BLK       164.0
TOV      3262.0
PF       1777.0
PTS     24368.0
dtype: float64